## Reconstructing Option RICs

## Overview

This notebook presents functions to construct option RICs on equities and indices traded in the following exchanges:
* US OPRA - refer to RULES7, RULES2, RULES3, RULES4 in Workspace, and Guideline for strikes above 10000 in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/datanotification/DN099473.html).
* EUREX - refer to RULES2, RULES3, RULES4 in Workspace, and general option RIC structure in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/faqs/2016/09/000195632.html). 
* Osaka Exchange - refer to RULES2, RULES3, RULES4 in Workspace, and RIC structure for Osaka exchange in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/faqs/2014/10/000189842.html).
* Stock Exchange of Hong Kong - refer to RULES2, RULES3, RULES4 in Workspace, and RIC structure for HK exchange in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/faqs/2021/04/000198505.html).
* Hong Kong Future Exchange - refer to RULES2, RULES3, RULES4 in Workspace, and RIC structure for HK exchange in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/faqs/2021/04/000198505.html).
* Intercontinental Exchange (ICE) - refer to RULES2, RULES3, RULES4 in Workspace, and general option RIC structure in [MyRefinitiv](https://my.refinitiv.com/content/mytr/en/faqs/2016/09/000195632.html). 

The syntax for the expired options is universal across exchanges and can be found [here](https://my.refinitiv.com/content/mytr/en/faqs/2018/09/000178972.html).

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 

----

To start, we first import the necessary packages. We use the [Data Libraries for Python](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) to retrieve the data.

In [1]:
import refinitiv.data as rd
import pandas as pd
from refinitiv.data.errors import RDError
from datetime import timedelta
from datetime import datetime

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fcd1b7434c0 {name='codebook'}>

## Helper functions

Before defining the core functions for RIC search let's define several helper functions which will be further used by the main functions.

#### Function to get exchanges where the option is traded

This function allows getting the list of exchange codes where an option on the given asset is traded.

In [3]:
def get_exchange_code(asset):
    
    response = rd.discovery.search(
        query = asset,
        filter = "SearchAllCategory eq 'Options' and Periodicity eq 'Monthly' ",
        select = 'ExchangeCode',
        group_by = "ExchangeCode",
    )
    exchanges = response.drop_duplicates()["ExchangeCode"].to_list()
    exchange_codes = []
    for exchange in exchanges:
        exchange_codes.append(exchange)
        
    return exchange_codes

In [6]:
get_exchange_code('VOD.L')

['EUX', 'IEU', 'CDE']

#### Function to get option expiration month code

This function takes the expiration date and option type as an input and returns the expiration month code for the option, which is further used to construct RICs

In [7]:
def get_exp_month(maturity, opt_type, strike = None, opra=False):
    
    maturity = pd.to_datetime(maturity)
    # define option expiration identifiers
    ident = {'1': {'exp': 'A','C': 'A', 'P': 'M'}, 
           '2': {'exp': 'B', 'C': 'B', 'P': 'N'}, 
           '3': {'exp': 'C', 'C': 'C', 'P': 'O'}, 
           '4': {'exp': 'D', 'C': 'D', 'P': 'P'},
           '5': {'exp': 'E', 'C': 'E', 'P': 'Q'},
           '6': {'exp': 'F', 'C': 'F', 'P': 'R'},
           '7': {'exp': 'G', 'C': 'G', 'P': 'S'}, 
           '8': {'exp': 'H', 'C': 'H', 'P': 'T'}, 
           '9': {'exp': 'I', 'C': 'I', 'P': 'U'}, 
           '10': {'exp': 'J', 'C': 'J', 'P': 'V'},
           '11': {'exp': 'K', 'C': 'K', 'P': 'W'}, 
           '12': {'exp': 'L', 'C': 'L', 'P': 'X'}}
    
    # get expiration month code for a month
    if opt_type.upper() == 'C':
        exp_month = ident[str(maturity.month)]['C']
        
    elif opt_type.upper() == 'P':
        exp_month = ident[str(maturity.month)]['P']
    
    if opra and strike > 999.999:
        exp_month = exp_month.lower()
            
    return ident, exp_month

In [8]:
ident, exp_month = get_exp_month('2022-06-03', 'P', strike = 100, opra = True)
exp_month

'R'

#### Function ro check for expiry

This function checks if the option is expired or not and adds the expiration numenclature if it is.

In [9]:
def check_expiry(ric, maturity, ident):
    maturity = pd.to_datetime(maturity)
    if maturity < datetime.now():
        ric = ric + '^' + ident[str(maturity.month)]['exp'] + str(maturity.year)[-2:]
    return ric

In [10]:
check_expiry('AAPLA212216000.U', '2022-01-21', ident)

'AAPLA212216000.U^A22'

#### Function to request prices

This function allows validation of the constructed option RICs by requesting prices. If prices are returned, we can confirm that the RIC(s) is (are) valid, otherwise, we can't confirm the validation.

In [11]:
def request_prices(ric):
    prices = []
    try:    
        prices = rd.get_history(ric, fields = ['BID','ASK','TRDPRC_1','SETTLE'])
    except RDError as err:
        print(f'Constructed ric {ric} -  {err}')
    return prices

In [12]:
request_prices('AAPLA212216000.U^A22')

AAPLA212216000.U^A22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2021-12-23,17.15,17.45,17.36,<NA>
2021-12-27,20.9,21.0,21.0,<NA>
2021-12-28,19.95,20.3,19.95,<NA>
2021-12-29,19.85,20.05,20.05,<NA>
2021-12-30,18.5,19.0,18.9,<NA>
2021-12-31,17.95,18.75,18.2,<NA>
2022-01-03,22.0,22.25,22.15,<NA>
2022-01-04,19.85,20.2,20.07,<NA>
2022-01-05,15.35,15.8,15.57,<NA>


## Functions to reconstruct RICs for different exchanges

Now, we have all our helper functions defined, let's move into introducing RIC reconstruction functions per eschange.

#### Function for OPRA

The below function returns option RICs for OPRA exchange.

In [13]:
def get_ric_opra(asset, maturity, strike, opt_type):
    maturity = pd.to_datetime(maturity)
    
    # trim underlying asset's RIC to get the required part for option RIC
    if asset[0] == '.': # check if the asset is an index or an equity
        asset_name = asset[1:] # get the asset name - we remove "." symbol for index options
    else:
        asset_name = asset.split('.')[0] # we need only the first part of the RICs for equities
        
    ident, exp_month = get_exp_month(maturity, opt_type, strike = strike, opra=True)

    # get strike prrice
    if type(strike) == float:
        int_part = int(strike)
        dec_part = str(str(strike).split('.')[1])
    else:
        int_part = int(strike)
        dec_part = '00'
    if len(dec_part) == 1:
        dec_part = dec_part + '0'

    if int(strike) < 10:
        strike_ric = '00' + str(int_part) + dec_part
    elif int_part >= 10 and int_part < 100:
        strike_ric = '0' + str(int_part) + dec_part
    elif int_part >= 100 and int_part < 1000:
        strike_ric = str(int_part) + dec_part
    elif int_part >= 1000 and int_part < 10000:
        strike_ric = str(int_part) + '0'
    elif int_part >= 10000 and int_part < 20000:
        strike_ric = 'A' + str(int_part)[-4:]
    elif int_part >= 20000 and int_part < 30000:
        strike_ric = 'B' + str(int_part)[-4:]      
    elif int_part >= 30000 and int_part < 40000:
        strike_ric = 'C' + str(int_part)[-4:]
    elif int_part >= 40000 and int_part < 50000:
        strike_ric = 'D' + str(int_part)[-4:]
        
    # build ric
    ric = asset_name + exp_month + str(maturity.day) + str(maturity.year)[-2:] + strike_ric + '.U'
    ric = check_expiry(ric, maturity, ident)
    
    prices = request_prices(ric)
    
    # return valid ric(s)
    if len(prices) == 0:
        print('RIC with specified parameters is not found')

    return ric, prices

In [14]:
ric, prices = get_ric_opra('AAPL.O', '2022-01-21', 160, 'P')

In [15]:
ric

'AAPLM212216000.U^A22'

In [16]:
prices

AAPLM212216000.U^A22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2021-12-23,0.9,0.98,0.94,<NA>
2021-12-27,0.54,0.55,0.55,<NA>
2021-12-28,0.55,0.57,0.56,<NA>
2021-12-29,0.48,0.5,0.48,<NA>
2021-12-30,0.44,0.5,0.46,<NA>
2021-12-31,0.38,0.46,0.44,<NA>
2022-01-03,0.21,0.23,0.23,<NA>
2022-01-04,0.27,0.28,0.29,<NA>
2022-01-05,0.58,0.7,0.63,<NA>


#### Function for the Stock Exchange of Hong Kong

The below function returns option RICs for Hong Kong exchange.

In [17]:
def get_ric_hk(asset, maturity, strike, opt_type):
    maturity = pd.to_datetime(maturity)
    
    # get asset name and strike price for the asset
    if asset[0] == '.': 
        asset_name = asset[1:] 
        strike_ric = str(int(strike))
    else:
        asset_name = asset.split('.')[0]
        strike_ric = str(int(strike * 100))
     
    # get expiration month codes
    ident, exp_month = get_exp_month(maturity, opt_type)

    # get rics for options on indexes. Return if valid add to the possible_rics list if no price is found
    if asset[0] == '.':
        ric = asset_name + strike_ric + exp_month + str(maturity.year)[-1:] + '.HF'
        ric = check_expiry(ric, maturity, ident)
        prices = request_prices(ric)
        
        if len(prices) == 0:
            print('RIC with specified parameters is not found')
        else:
            return ric, prices
    else:
        # get rics for options on equities. Return if valid add to the possible_rics list if no price is found
        # there could be several generations of options depending on the number of price adjustments due to a corporate event
        # here we use 4 adjustment opportunities.
        for i in range(4): 
            ric = asset_name + strike_ric + str(i)+ exp_month + str(maturity.year)[-1:] + '.HK'
            ric = check_expiry(ric, maturity, ident)
            prices = request_prices(ric)
            if len(prices) == 0:
                print('RIC with specified parameters is not found')
            else:
                return ric, prices
    return ric, prices

In [18]:
ric, prices = get_ric_hk('1093.HK', '2022-03-30', 10, 'C')

In [19]:
ric

'109310000C2.HK^C22'

In [20]:
prices

109310000C2.HK^C22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2022-03-03,0.05,0.08,<NA>,0.07
2022-03-04,0.04,0.1,<NA>,0.07
2022-03-07,0.04,0.07,0.05,0.04
2022-03-08,0.02,0.12,<NA>,0.07
2022-03-09,0.02,0.08,<NA>,0.05
2022-03-10,0.02,0.05,0.04,0.04
2022-03-11,0.02,0.06,0.03,0.03
2022-03-14,0.02,<NA>,0.03,0.02
2022-03-15,0.02,0.05,<NA>,0.02


#### Function for the Osaka Stock Exchange

The below function returns option RICs for Osaka exchange.

In [21]:
def get_ric_ose(asset, maturity, strike, opt_type):
    
    maturity = pd.to_datetime(maturity)
    strike_ric = str(strike)[:3] 
    ident, exp_month = get_exp_month(maturity, opt_type)
    
    j_nets = ['', 'L', 'R']
    generations = ['Y', 'Z', 'A', 'B', 'C']
    
    if asset[0] == '.':
        index_dict = {'N225':'JNI', 'TOPX':'JTI'}
        # Option Root codes for indexes are different from the RIC, so we rename where necessery
        asset_name = index_dict[asset.split('.')[1]]
        
        # we consider also J-NET (Off-Auction(with "L")) and High  frequency (with 'R') option structures 
        for jnet in j_nets:
            ric = asset_name + jnet + strike_ric + exp_month + str(maturity.year)[-1:] + '.OS'
            ric = check_expiry(ric, maturity, ident)
            prices = request_prices(ric)
            if len(prices) == 0:
                print('RIC with specified parameters is not found')
            else:
                return ric, prices
    else:
        asset_name = asset.split('.')[0]
         # these are generation codes similar to one from HK 
        for jnet in j_nets:
            for gen in generations:
                ric = asset_name + jnet + gen + strike_ric + exp_month + str(maturity.year)[-1:] + '.OS'
                ric = check_expiry(ric, maturity, ident)
                prices = request_prices(ric)
                if len(prices) == 0:
                    print('RIC with specified parameters is not found')
                else:
                    return ric, prices
    return ric, prices

In [22]:
ric, prices = get_ric_ose('7974.T', '2022-03-30', 50000, 'C')

In [23]:
ric

'7974Y500C2.OS^C22'

In [24]:
prices

7974Y500C2.OS^C22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2022-02-08,<NA>,<NA>,<NA>,8290
2022-02-09,<NA>,<NA>,<NA>,8615
2022-02-10,<NA>,<NA>,<NA>,8615
2022-02-14,<NA>,<NA>,<NA>,8385
2022-02-15,<NA>,<NA>,<NA>,8035
2022-02-16,<NA>,<NA>,<NA>,9005
2022-02-17,<NA>,<NA>,<NA>,9140
2022-02-18,<NA>,<NA>,<NA>,9760
2022-02-21,<NA>,<NA>,<NA>,9005


#### Function for the EUREX

Below function returns option RICs for EUREX exchange.

In [25]:
def get_ric_eurex(asset, maturity, strike, opt_type):
    maturity = pd.to_datetime(maturity)

    if asset[0] == '.': 
        index_dict = {'FTSE':'OTUK', 'SSMI':'OSMI', 'GDAXI':'GDAX', 'ATX':'FATXA', 'STOXX50E':'STXE'}
        asset_name = index_dict[asset.split('.')[1]]
    else:
        asset_name = asset.split('.')[0]
        
    ident, exp_month = get_exp_month(maturity, opt_type)
        
    if type(strike) == float:
        int_part = int(strike)
        dec_part = str(str(strike).split('.')[1])[0]
    else:
        int_part = int(strike)
        dec_part = '0'      
    if len(str(int(strike))) == 1:
        strike_ric = '0' + str(int_part) + dec_part
    else:
        strike_ric = str(int_part) + dec_part
    
    generations = ['', 'a', 'b', 'c', 'd']
    for gen in generations:
        ric = asset_name + strike_ric  + gen + exp_month + str(maturity.year)[-1:] + '.EX'
        ric = check_expiry(ric, maturity, ident)
        prices = request_prices(ric)
        if len(prices) == 0:
            print('RIC with specified parameters is not found')
        else:
            return ric, prices
    return ric, prices

In [26]:
ric, prices = get_ric_eurex('.STOXX50E', '2022-03-30', 4200, 'P')

In [27]:
ric

'STXE42000O2.EX^C22'

In [28]:
prices

STXE42000O2.EX^C22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2022-02-21,246.2,256.5,250.0,250.4
2022-02-22,241.7,253.0,227.5,246.3
2022-02-23,245.1,259.9,246.0,253.3
2022-02-24,70.0,391.5,375.0,382.9
2022-02-25,247.1,263.3,271.0,252.8
2022-02-28,284.8,298.2,339.4,285.9
2022-03-01,423.3,443.8,384.7,433.7
2022-03-02,376.3,388.8,377.0,376.3
2022-03-03,70.0,477.5,385.0,464.1


#### Function for the Intercontinental Exchange

The below function returns option RICs for Intercontinental Exchange exchange.

In [29]:
def get_ric_ieu(asset, maturity, strike, opt_type):
    maturity = pd.to_datetime(maturity)
    
    if asset[0] == '.':
        index_dict = {'FTSE':'LFE'}
        asset_name = index_dict[asset.split('.')[1]]     
    else:
        asset_name = asset.split('.')[0] 
        
    ident, exp_month = get_exp_month(maturity, opt_type)

    if len(str(int(strike))) == 2:
        strike_ric = '0' + str(int(strike))
    else:
        strike_ric = str(int(strike))
        
    if type(strike) == float and len(str(int(strike))) == 1:
        int_part = int(strike)
        dec_part = str(str(strike).split('.')[1])[0]        
        strike_ric = '0' + str(int_part) + dec_part
    
    generations = ['', 'a', 'b', 'c', 'd']
    for gen in generations:
        ric = asset_name + strike_ric  + gen + exp_month + str(maturity.year)[-1:] + '.L'
        ric = check_expiry(ric, maturity, ident)
        prices = request_prices(ric)
        if len(prices) == 0:
            print('RIC with specified parameters is not found')
        else:
            return ric, prices
    return  ric, prices

In [30]:
ric, prices = get_ric_ieu('.FTSE', '2022-06-30', 7000, 'C')

In [31]:
ric

'LFE7000F2.L^F22'

In [32]:
prices

LFE7000F2.L^F22,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2022-05-23,<NA>,<NA>,<NA>,530.5
2022-05-24,518,537.5,<NA>,510.0
2022-05-25,<NA>,<NA>,<NA>,551.5
2022-05-26,<NA>,<NA>,<NA>,584.5
2022-05-27,<NA>,<NA>,<NA>,596.5
2022-05-30,<NA>,<NA>,<NA>,604.0
2022-05-31,<NA>,<NA>,<NA>,598.5
2022-06-01,<NA>,<NA>,<NA>,542.0
2022-06-02,<NA>,<NA>,<NA>,542.0


#### Universal function for all above exchanges

Below is a unicersak function takes RIC, maturity, strike and option type as an input, finds all exchanges where the options on the given asset are traded, constructs RICs for them, validates and returns the constructed RICs along with the prices. Here again, If no price is found for constructed RICs, the functions print out possible RICs.

In [33]:
def get_option_ric(asset, maturity, strike, opt_type):
    
    # define covered exchanges along with functions to get RICs from
    exchanges = {'OPQ': get_ric_opra,
           'IEU': get_ric_ieu,
           'EUX': get_ric_eurex,
           'HKG': get_ric_hk,
           'HFE': get_ric_hk,
           'OSA': get_ric_ose}
    
    # get exchanges codes where the option on the given asset is traded
    exchnage_codes = get_exchange_code(asset)
    # get the list of (from all available and covered exchanges) valid rics and their prices
    options_data = {}
    for exch in exchnage_codes:
        if exch in exchanges.keys():
            ric, prices = exchanges[exch](asset, maturity, strike, opt_type)
            if len(prices) != 0:
                options_data[ric] = prices
                print(f'Option RIC for {exch} exchange is successfully constructed')     
        else:
            print(f'The {exch} exchange is not supported yet')
    return options_data

Below, I test the fuinction for several exchanges and assets:

In [34]:
# options_data = get_option_ric('ABBN.S', '2022-03-30', 34, 'C')
# options_data = get_option_ric('ALVG.DE', '2022-03-18', 220, 'C')
# options_data = get_option_ric('BBVA.MC', '2022-03-18', 4.7, 'C')
# options_data = get_option_ric('AIRP.PA', '2022-03-18', 150, 'C')
# options_data = get_option_ric('BARC.L', '2022-03-18', 210, 'P')
# options_data = get_option_ric('AZN.L', '2022-03-18', 9000, 'P')
# options_data = get_option_ric('VOD.L', '2022-03-18', 100, 'P')
# options_data = get_option_ric('ALSO.PA', '2022-06-18', 36, 'P')
# options_data = get_option_ric('ENI.MI', '2022-06-16', 13, 'C')
# options_data = get_option_ric('ASML.AS', '2022-02-18', 640, 'P')
# options_data = get_option_ric('.HSI', '2022-03-30', 18400, 'C')
# options_data = get_option_ric('1093.HK', '2022-03-30', 10, 'C')
# options_data = get_option_ric('0700.HK', '2022-04-28', 480, 'C')
# options_data = get_option_ric('.TOPX', '2022-06-10', 1900, 'C')
# options_data = get_option_ric('6501.T', '2022-06-10', 6500, 'C')
# options_data = get_option_ric('.STOXX50E', '2022-03-18', 4200, 'C')
# options_data = get_option_ric('.N225', '2022-01-17', 25875, 'C')
options_data = get_option_ric('.SPX', '2022-02-18', 5000, 'C')

Option RIC for OPQ exchange is successfully constructed


In [35]:
options_data

{'SPXb182250000.U^B22': SPXb182250000.U^B22   BID   ASK  TRDPRC_1  SETTLE
 Date                                             
 2022-01-21           0.25   0.6       0.5    <NA>
 2022-01-24            0.5   0.7      0.65    <NA>
 2022-01-25            0.4   0.6      0.53    <NA>
 2022-01-26            0.4   0.8      0.75    <NA>
 2022-01-27            0.3   0.4       0.4    <NA>
 2022-01-28           0.45  0.65       0.6    <NA>
 2022-01-31            0.2   0.3      0.25    <NA>
 2022-02-01            0.2   0.3      0.23    <NA>
 2022-02-02           0.05  0.25      0.25    <NA>
 2022-02-03           0.05  0.15      0.15    <NA>
 2022-02-04           0.05   0.1       0.1    <NA>
 2022-02-07           <NA>   0.1      0.05    <NA>
 2022-02-08           <NA>   0.1      0.08    <NA>
 2022-02-09           <NA>  0.15      0.08    <NA>
 2022-02-10           <NA>  0.05      0.03    <NA>
 2022-02-11           <NA>  0.15      <NA>    <NA>
 2022-02-14           <NA>  0.05      0.05    <NA>
 2022-02

In [36]:
rd.close_session()